# Import Library

In [1]:
import pandas as pd
import numpy as np
import string
import re #regrex libray
import nltk
import swifter
import Sastrawi
import networkx as nx

from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# Load Dataset

In [2]:
data_berita = pd.read_csv('berita2.csv')
data_berita.head()

,judul,tanggal,kategori,deskripsi
0,Begini Cara Menghemat Baterai iPhone,"Jum'at, 22 April 2022 - 19:53 WIB",TELCO,Cara menghemat baterai iPhone rupanya bisa dil...
1,3 Cara Menghilangkan Virus di HP Android denga...,"Jum'at, 22 April 2022 - 19:20 WIB",TELCO,Sebagian orang merasa bingung mencari cara men...
2,Cara Membuat PPT di HP dengan Microsoft,"Jum'at, 22 April 2022 - 19:01 WIB",TELCO,"Bagi anda yang sering melakukan presentasi, pe..."
3,Bug di Google Messages Bikin Baterai Boros dan...,"Jum'at, 22 April 2022 - 17:50 WIB",TELCO,Bug pada aplikasi Google Messages berhasil dii...
4,Sony Siap Tayangkan Iklan di Game PlayStation,"Jum'at, 22 April 2022 - 17:40 WIB",GAMES,Sony dilaporkan berencana menyematkan iklan pa...


# Case Folding

In [3]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data_berita['deskripsi'] = data_berita['deskripsi'].str.lower()


print('Case Folding Result : \n')
print(data_berita['deskripsi'].head(20))
print('\n\n\n')

Case Folding Result : 

0     cara menghemat baterai iphone rupanya bisa dil...
1     sebagian orang merasa bingung mencari cara men...
2     bagi anda yang sering melakukan presentasi, pe...
3     bug pada aplikasi google messages berhasil dii...
4     sony dilaporkan berencana menyematkan iklan pa...
5     lebaran sebentar lagi. seperti biasa, pada mom...
6     banyak smartphone yang dibandrol dengan harga ...
7     oppo a96 resmi melantai di indonesia, kamis (7...
8     salah satu yang cocok jadi teman ngabuburit ad...
9     pt. lg electronics indonesia (lg) resmi memasa...
10    , google doodle secara jelas memperlihatkan ga...
11    hai sobat gamer! bermain game sambil mengatur ...
12    samsung galaxy a73 5g adalah varian tertinggi ...
13    institusi pemerintah indonesia menjadi jawara ...
14    elon musk makin getol ingin mengambil alih twi...
15    kode redeem ff free fire jumat 22 april 2022, ...
16    bocoran terkait iphone 16 sudah mulai beredar ...
17    cara cek imei hp s

# Tokenizing

In [4]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data_berita['deskripsi'] = data_berita['deskripsi'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_berita['deskripsi_tokens'] = data_berita['deskripsi'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data_berita['deskripsi_tokens'].head(20))
print('\n\n\n')

Tokenizing Result : 

0     [cara, menghemat, baterai, iphone, rupanya, bi...
1     [sebagian, orang, merasa, bingung, mencari, ca...
2     [bagi, anda, yang, sering, melakukan, presenta...
3     [bug, pada, aplikasi, google, messages, berhas...
4     [sony, dilaporkan, berencana, menyematkan, ikl...
5     [lebaran, sebentar, lagi, seperti, biasa, pada...
6     [banyak, smartphone, yang, dibandrol, dengan, ...
7     [oppo, resmi, melantai, di, indonesia, kamis, ...
8     [salah, satu, yang, cocok, jadi, teman, ngabub...
9     [pt, lg, electronics, indonesia, lg, resmi, me...
10    [google, doodle, secara, jelas, memperlihatkan...
11    [hai, sobat, gamer, bermain, game, sambil, men...
12    [samsung, galaxy, adalah, varian, tertinggi, d...
13    [institusi, pemerintah, indonesia, menjadi, ja...
14    [elon, musk, makin, getol, ingin, mengambil, a...
15    [kode, redeem, ff, free, fire, jumat, april, b...
16    [bocoran, terkait, iphone, sudah, mulai, bered...
17    [cara, cek, imei, hp

# Menghitung Frekuensi Distribusi Token

In [5]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data_berita['deskripsi_tokens_fdist'] = data_berita['deskripsi_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data_berita['deskripsi_tokens_fdist'].head(20).apply(lambda x : x.most_common()))

Frequency Tokens : 

0     [(cara, 2), (dengan, 2), (yang, 2), (menghemat...
1     [(sebagian, 1), (orang, 1), (merasa, 1), (bing...
2     [(anda, 2), (membuat, 2), (ppt, 2), (bagi, 1),...
3     [(bug, 2), (aplikasi, 2), (pada, 1), (google, ...
4     [(game, 3), (iklan, 2), (pada, 2), (sony, 1), ...
5     [(yang, 4), (lebaran, 1), (sebentar, 1), (lagi...
6     [(yang, 3), (dengan, 3), (harga, 2), (banyak, ...
7     [(oppo, 3), (yang, 2), (resmi, 1), (melantai, ...
8     [(satu, 2), (yang, 2), (salah, 1), (cocok, 1),...
9     [(tv, 4), (lg, 2), (premium, 2), (led, 2), (pt...
10    [(google, 1), (doodle, 1), (secara, 1), (jelas...
11    [(game, 2), (hai, 1), (sobat, 1), (gamer, 1), ...
12    [(samsung, 2), (di, 2), (galaxy, 1), (adalah, ...
13    [(institusi, 1), (pemerintah, 1), (indonesia, ...
14    [(elon, 1), (musk, 1), (makin, 1), (getol, 1),...
15    [(free, 2), (fire, 2), (kode, 1), (redeem, 1),...
16    [(bocoran, 2), (terkait, 1), (iphone, 1), (sud...
17    [(cara, 1), (cek, 1),

# Filtering (Stopword Removal)

In [6]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("berita.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_berita['deskripsi_tokens_WSW'] = data_berita['deskripsi_tokens'].apply(stopwords_removal) 


print(data_berita['deskripsi_tokens_WSW'].head(20))

0     [menghemat, baterai, iphone, mudah, mode, daya...
1     [orang, bingung, mencari, menghilangkan, virus...
2                [presentasi, ppt, hp, ppt, smartphone]
3     [bug, aplikasi, google, messages, berhasil, di...
4     [sony, dilaporkan, berencana, menyematkan, ikl...
5     [lebaran, sebentar, momen, spesial, cair, nama...
6     [smartphone, dibandrol, harga, selangit, diban...
7     [oppo, resmi, melantai, indonesia, kamis, smar...
8     [salah, cocok, teman, ngabuburit, oppo, pasaln...
9     [pt, lg, electronics, indonesia, lg, resmi, me...
10    [google, doodle, gambar, berformat, gif, dampa...
11    [hai, sobat, gamer, bermain, game, mengatur, l...
12    [samsung, galaxy, varian, tertinggi, seri, sam...
13    [institusi, pemerintah, indonesia, jawara, keb...
14    [elon, musk, getol, mengambil, alih, twitter, ...
15    [kode, redeem, ff, free, fire, jumat, april, m...
16    [bocoran, terkait, iphone, beredar, intenet, b...
17    [cek, imei, hp, samsung, sulit, pengguna, 

# Normalization

In [7]:
normalizad_word = pd.read_csv("berita.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data_berita['deskripsi_normalized'] = data_berita['deskripsi_tokens_WSW'].apply(normalized_term)

data_berita['deskripsi_normalized'].head(20)

0     [menghemat, baterai, iphone, mudah, mode, daya...
1     [orang, bingung, mencari, menghilangkan, virus...
2                [presentasi, ppt, hp, ppt, smartphone]
3     [bug, aplikasi, google, messages, berhasil, di...
4     [sony, dilaporkan, berencana, menyematkan, ikl...
5     [lebaran, sebentar, momen, spesial, cair, nama...
6     [smartphone, dibandrol, harga, selangit, diban...
7     [oppo, resmi, melantai, indonesia, kamis, smar...
8     [salah, cocok, teman, ngabuburit, oppo, pasaln...
9     [pt, lg, electronics, indonesia, lg, resmi, me...
10    [google, doodle, gambar, berformat, gif, dampa...
11    [hai, sobat, gamer, bermain, game, mengatur, l...
12    [samsung, galaxy, varian, tertinggi, seri, sam...
13    [institusi, pemerintah, indonesia, jawara, keb...
14    [elon, musk, getol, mengambil, alih, twitter, ...
15    [kode, redeem, ff, free, fire, jumat, april, m...
16    [bocoran, terkait, iphone, beredar, intenet, b...
17    [cek, imei, hp, samsung, sulit, pengguna, 

# Stemming

In [8]:

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_berita['deskripsi_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_berita['deskripsi_tokens_stemmed'] = data_berita['deskripsi_normalized'].swifter.apply(get_stemmed_term)
print(data_berita['deskripsi_tokens_stemmed'])

207
------------------------
menghemat : hemat
baterai : baterai
iphone : iphone
mudah : mudah
mode : mode
daya : daya
rendah : rendah
didengar : dengar
orang : orang
bingung : bingung
mencari : cari
menghilangkan : hilang
virus : virus
hp : hp
android : android
mengikuti : ikut
petunjuk : tunjuk
sesuai : sesuai
presentasi : presentasi
ppt : ppt
smartphone : smartphone
bug : bug
aplikasi : aplikasi
google : google
messages : messages
berhasil : hasil
diidentifikasi : identifikasi
laporan : lapor
togoogle : togoogle
kamera : kamera
berjalan : jalan
sony : sony
dilaporkan : lapor
berencana : rencana
menyematkan : semat
iklan : iklan
game : game
playstation : playstation
dimainkan : main
gratis : gratis
muncul : muncul
keuntungan : untung
pengembang : kembang
lebaran : lebaran
sebentar : sebentar
momen : momen
spesial : spesial
cair : cair
namanya : nama
tunjangan : tunjang
raya : raya
thr : thr
berniat : niat
membeli : beli
direkomendasikan : rekomendasi
dibandrol : dibandrol
harga : har

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

0     [hemat, baterai, iphone, mudah, mode, daya, re...
1     [orang, bingung, cari, hilang, virus, hp, andr...
2                [presentasi, ppt, hp, ppt, smartphone]
3     [bug, aplikasi, google, messages, hasil, ident...
4     [sony, lapor, rencana, semat, iklan, game, pla...
5     [lebaran, sebentar, momen, spesial, cair, nama...
6     [smartphone, dibandrol, harga, langit, bandero...
7     [oppo, resmi, lantai, indonesia, kamis, smartp...
8     [salah, cocok, teman, ngabuburit, oppo, pasal,...
9     [pt, lg, electronics, indonesia, lg, resmi, pa...
10    [google, doodle, gambar, format, gif, dampak, ...
11    [hai, sobat, gamer, main, game, atur, lintas, ...
12    [samsung, galaxy, varian, tinggi, seri, samsun...
13    [institusi, perintah, indonesia, jawara, bocor...
14    [elon, musk, getol, ambil, alih, twitter, dana...
15    [kode, redeem, ff, free, fire, jumat, april, m...
16    [bocor, kait, iphone, edar, intenet, bocor, ka...
17          [cek, imei, hp, samsung, sulit, guna

# Simpan Data ke CSV

In [9]:
data_berita.to_csv("Text_Preprocessing2.csv")

# Prepare Data

In [10]:
import pandas as pd 
import numpy as np

data_berita = pd.read_csv("Text_Preprocessing2.csv", usecols=["deskripsi_tokens_stemmed"])
data_berita.columns = ["deskripsi"]

data_berita.head(20)

,deskripsi
0,"['hemat', 'baterai', 'iphone', 'mudah', 'mode'..."
1,"['orang', 'bingung', 'cari', 'hilang', 'virus'..."
2,"['presentasi', 'ppt', 'hp', 'ppt', 'smartphone']"
3,"['bug', 'aplikasi', 'google', 'messages', 'has..."
4,"['sony', 'lapor', 'rencana', 'semat', 'iklan',..."
5,"['lebaran', 'sebentar', 'momen', 'spesial', 'c..."
6,"['smartphone', 'dibandrol', 'harga', 'langit',..."
7,"['oppo', 'resmi', 'lantai', 'indonesia', 'kami..."
8,"['salah', 'cocok', 'teman', 'ngabuburit', 'opp..."
9,"['pt', 'lg', 'electronics', 'indonesia', 'lg',..."


# Term Frekuensi

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

a=len(document)
document = data_berita['deskripsi']

# Create a Vectorizer Object
vectorizer = CountVectorizer()

vectorizer.fit(document)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(document)

# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

Vocabulary:  {'hemat': 64, 'baterai': 10, 'iphone': 75, 'mudah': 117, 'mode': 115, 'daya': 34, 'rendah': 152, 'dengar': 35, 'orang': 128, 'bingung': 15, 'cari': 22, 'hilang': 65, 'virus': 193, 'hp': 66, 'android': 4, 'ikut': 70, 'tunjuk': 183, 'sesuai': 163, 'presentasi': 138, 'ppt': 136, 'smartphone': 164, 'bug': 17, 'aplikasi': 5, 'google': 57, 'messages': 112, 'hasil': 63, 'identifikasi': 67, 'lapor': 101, 'togoogle': 175, 'kamera': 84, 'jalan': 77, 'sony': 166, 'rencana': 151, 'semat': 161, 'iklan': 68, 'game': 52, 'playstation': 134, 'main': 108, 'gratis': 58, 'muncul': 118, 'untung': 189, 'kembang': 88, 'lebaran': 102, 'sebentar': 159, 'momen': 116, 'spesial': 167, 'cair': 20, 'nama': 122, 'tunjang': 182, 'raya': 147, 'thr': 173, 'niat': 124, 'beli': 14, 'rekomendasi': 150, 'dibandrol': 38, 'harga': 62, 'langit': 99, 'banderol': 9, 'murah': 119, 'kisar': 92, 'rp': 154, 'juta': 81, 'fitur': 46, 'pukau': 142, 'oppo': 127, 'resmi': 153, 'lantai': 100, 'indonesia': 72, 'kamis': 85, '

In [12]:
a = vectorizer.get_feature_names()

# TF-IDF

In [13]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tf = tfidf.fit_transform(vectorizer.fit_transform(document)).toarray()

In [14]:
dfb = pd.DataFrame(data=tf,index=list(range(1, len(tf[:,1])+1, )),columns=[a])
dfb

,aiot,alih,ambil,analis,android,aplikasi,april,atur,baik,banderol,...,upaya,varian,versi,virus,wajib,warna,wujud,yuk,zakat,zaman
1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.31006,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.31006,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.503252,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.273801,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.214433,0.000000,0.000000,0.00000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
10,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000


# Convert File to CSV

In [15]:
dfb.to_csv("TF-IDF2.csv")